# DRY Examples

### Example of the simplest class

In [1]:

class account:
    pass 

account.debits = 50
account.credits = 100
account.fees = -2
account.balance = 175
    
print(dir(account))
    


['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'balance', 'credits', 'debits', 'fees']


### Another simple class

In [2]:
class person:
    def __init__(self):
        self.age = 8
        self.height = 4
        
    def get_age(self):
        return self.age

    def get_height(self):
        return self.height

In [3]:
Karla = person()

print(Karla.get_age())
print(Karla.get_height())

8
4


### DUPLICATION IN CODE 

It may be trivial, but code duplication is oh, so common. Here’s
an example:

In [4]:
def print_balance(account):
    print("Debits: \n\t", account.debits)
    print("Credits:\n\t", account.credits)
    if account.fees < 0:
        print("Fees: \n\t", "-" + str(account.fees))  
    else:
        print("Fees: \n\t", account.fees)
    
    print(" ———-\n")
    if account.balance < 0:
        print("Balance: \n\t", "-" + str(account.balance))
    else:
        print("Balance: \n\t", account.balance)
 

In [5]:
print_balance(account)

Debits: 
	 50
Credits:
	 100
Fees: 
	 --2
 ———-

Balance: 
	 175


For now ignore the implication that we’re committing the
newbie mistake of storing currencies in floats. Instead see if you
can spot duplications in this code. (We can see at least three
things, but you might see more.)
What did you find? Here’s our list.
First, there’s clearly a copy-and-paste duplication of handling
the negative numbers. We can fix that by adding another
function:


In [6]:
def format_amount(value):
    if value < 0:
        return "-" + str(value)
    else:
        return str(value)
    
def print_balance(account):
    print("Debits: \n\t", account.debits)
    print("Credits: \n\t", account.credits)
    print("Fees: \n\t", format_amount(account.fees))
    print(" ———\n")
    print("Balance: \n\t", format_amount(account.balance))

In [7]:
print_balance(account)

Debits: 
	 50
Credits: 
	 100
Fees: 
	 --2
 ———

Balance: 
	 175


Another duplication is the repetition of the field width in all the
printf calls. We could fix this by introducing a constant and
passing it to each call, but why not just use the existing
function?

In [8]:
def format_amount(value):
    if value < 0:
        return "\n\t" + "-" + str(value)
    else:
        return "\n\t" + str(value)

def print_balance(account):
    print("Debits: ", format_amount(account.debits))
    print("Credits: ", format_amount(account.credits))
    print("Fees: ", format_amount(account.fees))
    print(" ———-\n")
    print("Balance: ", format_amount(account.balance))

In [9]:
print_balance(account)

Debits:  
	50
Credits:  
	100
Fees:  
	--2
 ———-

Balance:  
	175


Anything more? Well, what if the client asks for an extra space
between the labels and the numbers? We’d have to change five
lines. Let’s remove that duplication:

In [10]:
def format_amount(value):
    if value < 0:
        return "\n\t" + "-" + str(value)
    else:
        return "\n\t" + str(value)

def print_line(label, value):
    print("\n", label, value)

def report_line(label, amount):
    print_line(label + ":", format_amount(amount))

def print_balance(account):
    report_line("Debits", account.debits)
    report_line("Credits", account.credits)
    report_line("Fees", account.fees)
    print_line("", "———-")
    report_line("Balance", account.balance)



In [11]:
print_balance(account)


 Debits: 
	50

 Credits: 
	100

 Fees: 
	--2

  ———-

 Balance: 
	175


# Extracting string in a source text file

    The main goal is start by implement a code that extracts string 
    (i.e. words in enclosed by " " ) from a textfile. Eventually we will
    design code that can do the same extraction of string but with other
    types of files, not just text. 

    A mathematician in the math division has designed this finite state 
    machine (FSM) that has the logic necessary for this implementation. 
    This FSM diagram is presented as follows:


<img src = "event_string_fsm.png"
        alt = "FSM String Extraction" width = 450 />
        
                                        
                                        Reference: Pragmatic Programmer 2nd edition

    First let us design the finite state machine table:


$$
\begin{array}{|l|c|c|c|}
\hline
 \ & \textbf{q_mark} & \textbf{back_slash } & \textbf{neither q_mark nor back_slash } \\ 
\hline
\textbf{looking_str} & \text{in_str} & \text{looking_str} & \text{looking_str} \\
\hline
\textbf{in_str} & \text{looking_str} & \text{skip_q_mark}& \text{in_str} \\
\hline
\textbf{skip_q_mark} & \text{in_str} & \text{in_str} & \text{in_str} \\
\hline
\end{array}
$$

### First the procedural implementation

In [12]:
def file_location_to_str(location):
    original_file = open(location,"r") 
    contents_list = original_file.read() #file to str
    original_file.close()
    return contents_list
    
    
def new_state(current_state = "looking_str", event = "neither"):
    machine_states = {"looking_str":["in_str", "looking_str", "looking_str"],
                      "in_str": ["looking_str", "skip_q_mark", "in_str"],
                      "skip_q_mark": ["in_str", "in_str", "in_str"]}
    
    activation_inputs = {"q_mark": 0, "back_slash": 1, "neither": 2}
    return machine_states[current_state][activation_inputs[event]]

if __name__ == "__main__":
    
    file_contents=file_location_to_str("/Users/luise/Desktop/Jupyter_Notebooks/extract_str_example.txt")
    print(file_contents)
    
    #starting state and event 
    current_state = "looking_str"
    event = "neither"
    
    new_str = ""
    file_strs = []
    transition = False
    
    for letter in file_contents:
        if letter == '"':
            event = "q_mark"
        elif letter == "\\":
            event = "back_slash"   
        else:
            event = "neither"

        prev_state = current_state
        transition = False
        
        current_state = new_state(current_state,event)
            
        if (prev_state,current_state) == ('looking_str', 'looking_str'):
            pass
        elif (prev_state,current_state) == ('looking_str', 'in_str'):
            pass
        elif (prev_state,current_state) == ('looking_str', 'skip_qmark'):
            pass
        elif (prev_state,current_state) == ('in_str', 'looking_str'):
            file_strs.append(new_str) #saving in list instead of output result as in diagram
            print(new_str)
            new_str=""
        elif (prev_state,current_state) == ('in_str', 'in_str'):
            new_str += letter
        elif (prev_state,current_state) == ('in_str', 'skip_qmark'):
            pass
        elif (prev_state,current_state) == ('skip_qmark', 'looking_str'):
            pass
        elif (prev_state,current_state) == ('skip_qmark', 'in_str'):
            pass
        elif (prev_state,current_state) == ('skip_qmark', 'skip_qmark'):
            pass
       
    print("\n",file_strs) 


while True:
    instruction = input("Please enter the assembly instruction: ")

    mnemonic = instruction.split(" ")[0]
    print("Mnemonic:",mnemonic)

    computers_ISA = {"LOD":"00010000" , "STO":"00010001",  "ADD":"00100000",
                     "SUB":"00100001",  "JMP":"00110000",  "HLT": "11111111"}           

    instructions_list = instruction.split(" ",1)
    print(instructions_list)
    registers = instructions_list[1].replace(" ","")
    print(registers)
    register_list = registers.split(",")

    num_reg1 = register_list[0][1:]  # i values of register Ri 
    num_reg2 = register_list[1][1:]  # will be used in elifs below
    num_reg3 = register_list[2][1:]

    if mnemonic not in computers_ISA:
        print("There was a problem with the instruction mnemonic. Please consult the ISA documentation.\n\n")
        
    else:
        if len(register_list) != 3:
            print("Few or to many registers! Please, try again.\n\n")

        elif not (register_list[0][0] == re

# Data Pipeline with exceptions: Extracting from a file with Regular Expressions



$$\begin{array}{|l|c|c|c|}
\hline
\text{Functions in the pipeline} & \text{Exception}//
\hline
//
//
//
//
\end{array}


